<a href="https://colab.research.google.com/github/paridaabhishek/Alpacatrade/blob/master/5MinsWithTimeZoneLiveProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Install WebSocket
!pip install websocket-client
!pip install alpaca-trade-api

In [3]:
#Import Libreris
import websocket 
import json
from datetime import datetime
import pandas as pd
import numpy as np
from IPython.core.display import JSON
import pytz
from datetime import datetime
from datetime import timedelta

In [4]:
#Map Google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
ticket_interested = (input('Enter The Stock yo wanted to autotrade')).upper()
print(ticket_interested+ ' Will be autotraded now!! Sitback and enjoy your day!!')

listen_message='{"action":"subscribe","bars":["'+ticket_interested+'"]}'
print(listen_message)

Enter The Stock yo wanted to autotradeaapl
AAPL Will be autotraded now!! Sitback and enjoy your day!!
{"action":"subscribe","bars":["AAPL"]}


In [6]:
#Import Credentials. Kept the credential in the file Cred.py under the dir below for security.
import sys
import os
import glob
print(glob.glob("/content/drive/*"))
os.chdir('/content/drive//MyDrive/AlpecaTrading/Config')
#sys.path.append('/content/drive/AlpecaTrading/Config')
print(os.getcwd())
from  Cred import *

['/content/drive/MyDrive']
/content/drive/MyDrive/AlpecaTrading/Config


In [7]:
#Websocket Code
socket = "wss://stream.data.alpaca.markets/v2/sip" #Webscoket given by alpeca for 
#socket = "wss://paper-api.alpaca.markets/stream" #Webscoket given by alpeca for 
#socket = "wss://stream.data.alpaca.markets/v1beta1/crypto"

print(socket.split('/')[-1])

def on_ope_fun(ws):
    print("Opened")
    auth_data = {"action": "auth","key": API_KEY,"secret": SECRET_KEY }  #Key and Secret getting from the Cred file. 
    ws.send(json.dumps(auth_data))
    #listen_message={"action":"subscribe","trades":["BTCUSD"],"quotes":["LTCUSD","ETHUSD"],"bars":["BCHUSD"]}
    #listen_message={"action":"subscribe","trades":["BTCUSD"]}
    #listen_message={"action":"subscribe","bars":["BTCUSD"]}
    listen_message='{"action":"subscribe","bars":["'+ticket_interested+'"]}'
    #print(listen_message)
    #listen_message={"action":"subscribe","bars":["AAPL"]}  ### --> Modify based on whatever symbol you want to check
    ws.send(json.dumps(json.loads(listen_message)))


    print('Still Open')

def on_msg_fun(ws,message):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("messgae received at"+ current_time)
    print(message)
    print('Strated wring to the steam file')
    OnetoFiveMinLive(message)
    print('Wrttin  steam file')

def on_clg_fun(ws):
    print("Closed")

sip


In [11]:
def OnetoFiveMinLive(message):
  try:

    #######################Create a file which will hold the stream data based on the ticker name ########################################################
    messageJson = json.loads(message)
    df = pd.DataFrame(messageJson)
    df.to_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),mode='a', index=False, header=False)

    with open('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested), 'r') as fp:
      for count, line in enumerate(fp):
        pass
      total_lines =count+1


    print('Total Lines in the Strimming file'+str(total_lines))

    if total_lines <= 10 :
       print('As we less than 10 and is :'+str(total_lines)+'. datapoints 1 to 5 mins bar calucation will be skipped till we have 10 points ....')


    if total_lines > 10:

      print('As we have 10 datapoints the calucation for 1 to 5 mins conversion started....')
      df = pd.read_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),skiprows=3,header=None,index_col=False)
      df.columns =['Message Type', 
                'Symbol', 
                'Open', 
                'Close',
                'High',
                'Low',
                'Volume',
                'TimeStamp',
                'n',
                'vw']
      df_unique=df.drop_duplicates(subset=['TimeStamp']).copy()
      df_unique.loc[:,'TimeStamp'] = pd.to_datetime(df_unique['TimeStamp'])
      df_unique.set_index('TimeStamp',inplace=True)
      print('Total Lines in the unique file is :'+ str(len(df_unique.index)))

      #Convert the UTC timeZone to easter timezone
      eastern = pytz.timezone('US/Central')
      df_unique.index = df_unique.index.tz_convert(eastern)
      df_unique.to_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/BarFiles/{}_1MinsBar.csv'.format(ticket_interested),mode='w', index=True, header=True)


      df_unique['5MinReminder'] = df_unique.index.minute % 5
      df_unique['15MinReminder'] = df_unique.index.minute % 15 #incase want to code for 15 mins bars

      min_time=df_unique[df_unique['5MinReminder']==0].index.min() #gets min time time where the 5 mins remider is zero. We need the data from next minute
      start_time = min_time + timedelta(minutes=1) # first pointer from where the 5 mins bar will start


      df_unique_start=df_unique.loc[start_time:].copy()

      df_5mins_bar=df_unique_start.groupby(pd.Grouper(freq='5min')).agg({'Open':'first',
                                                       'Close':'last',
                                                       'High':'max',
                                                       'Low':'min'}).iloc[1:]
      
      
      df_5mins_bar.to_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/BarFiles/{}_5MinsBar.csv'.format(ticket_interested),mode='w', index=True, header=True)
      print(df_5mins_bar)

     









  except Exception as ex:
    print(ex)








      





      





In [12]:
ws = websocket.WebSocketApp(socket, on_open=on_ope_fun,on_message=on_msg_fun)
ws.run_forever()

Opened
Still Open
messgae received at18:35:34
[{"T":"success","msg":"connected"}]
Strated wring to the steam file
Total Lines in the Strimming file1
As we less than 10 and is :1. datapoints 1 to 5 mins bar calucation will be skipped till we have 10 points ....
Wrttin  steam file
messgae received at18:35:35
[{"T":"success","msg":"authenticated"}]
Strated wring to the steam file
Total Lines in the Strimming file2
As we less than 10 and is :2. datapoints 1 to 5 mins bar calucation will be skipped till we have 10 points ....
Wrttin  steam file
messgae received at18:35:35
[{"T":"subscription","trades":[],"quotes":[],"bars":["AAPL"],"dailyBars":[],"statuses":[],"lulds":[],"corrections":[],"cancelErrors":[]}]
Strated wring to the steam file
Total Lines in the Strimming file3
As we less than 10 and is :3. datapoints 1 to 5 mins bar calucation will be skipped till we have 10 points ....
Wrttin  steam file
messgae received at18:36:00
[{"T":"b","S":"AAPL","o":170.59,"c":170.58,"h":170.66,"l":170.

False

In [ ]:
#########################Testing 
df = pd.read_csv('/content/drive//MyDrive/AlpecaTrading/DataFiles/StreamFiles/{}.csv'.format(ticket_interested),skiprows=3,header=None,index_col=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       10 non-null     object 
 1   1       10 non-null     object 
 2   2       10 non-null     float64
 3   3       10 non-null     float64
 4   4       10 non-null     float64
 5   5       10 non-null     float64
 6   6       10 non-null     int64  
 7   7       10 non-null     object 
 8   8       10 non-null     int64  
 9   9       10 non-null     float64
dtypes: float64(5), int64(2), object(3)
memory usage: 928.0+ bytes


In [ ]:
df.columns =['Message Type', 
                'Symbol', 
                'Open', 
                'Close',
                'High',
                'Low',
                'Volume',
                'TimeStamp',
                'n',
                'vw']

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Message Type  10 non-null     object 
 1   Symbol        10 non-null     object 
 2   Open          10 non-null     float64
 3   Close         10 non-null     float64
 4   High          10 non-null     float64
 5   Low           10 non-null     float64
 6   Volume        10 non-null     int64  
 7   TimeStamp     10 non-null     object 
 8   n             10 non-null     int64  
 9   vw            10 non-null     float64
dtypes: float64(5), int64(2), object(3)
memory usage: 928.0+ bytes


In [ ]:
df.loc[:,'TimeStamp'] = pd.to_datetime(df['TimeStamp']) 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Message Type  10 non-null     object             
 1   Symbol        10 non-null     object             
 2   Open          10 non-null     float64            
 3   Close         10 non-null     float64            
 4   High          10 non-null     float64            
 5   Low           10 non-null     float64            
 6   Volume        10 non-null     int64              
 7   TimeStamp     10 non-null     datetime64[ns, UTC]
 8   n             10 non-null     int64              
 9   vw            10 non-null     float64            
dtypes: datetime64[ns, UTC](1), float64(5), int64(2), object(2)
memory usage: 928.0+ bytes


In [ ]:
df_unique=df.drop_duplicates(subset=['TimeStamp']).copy()
df_unique.head(10)
# df_unique.loc[:,'TimeStamp'] = pd.to_datetime(df_unique['TimeStamp'])
# df_unique.set_index('TimeStamp',inplace=True)


,Message Type,Symbol,Open,Close,High,Low,Volume,TimeStamp,n,vw
0,b,MSFT,298.94,298.9800,299.006,298.9300,21606,2022-02-15 18:59:00+00:00,456,298.970346
2,b,MSFT,299.01,299.1000,299.105,298.9400,44096,2022-02-15 19:00:00+00:00,816,299.033441
3,b,MSFT,299.09,298.9400,299.090,298.8500,30277,2022-02-15 19:01:00+00:00,560,298.939871
4,b,MSFT,298.94,298.8000,298.940,298.6700,39193,2022-02-15 19:02:00+00:00,680,298.785090
5,b,MSFT,298.81,298.7000,298.910,298.6000,49387,2022-02-15 19:03:00+00:00,866,298.739768
7,b,MSFT,298.71,298.8200,298.870,298.6900,40825,2022-02-15 19:04:00+00:00,675,298.776884
8,b,MSFT,298.85,298.5650,298.880,298.4627,34817,2022-02-15 19:05:00+00:00,734,298.617803
9,b,MSFT,298.59,298.3801,298.630,298.3500,25911,2022-02-15 19:06:00+00:00,576,298.469432


In [ ]:
df_unique.loc[:,'TimeStamp'] = pd.to_datetime(df_unique['TimeStamp'])

In [ ]:
df_unique.set_index('TimeStamp',inplace=True)

In [ ]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8 entries, 2022-02-15 18:59:00+00:00 to 2022-02-15 19:06:00+00:00
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Message Type  8 non-null      object 
 1   Symbol        8 non-null      object 
 2   Open          8 non-null      float64
 3   Close         8 non-null      float64
 4   High          8 non-null      float64
 5   Low           8 non-null      float64
 6   Volume        8 non-null      int64  
 7   n             8 non-null      int64  
 8   vw            8 non-null      float64
dtypes: float64(5), int64(2), object(2)
memory usage: 640.0+ bytes


In [ ]:
eastern = pytz.timezone('US/Central')



In [ ]:
df_unique.index = df_unique.index.tz_convert(eastern)

In [ ]:
df_unique

,Message Type,Symbol,Open,Close,High,Low,Volume,n,vw
TimeStamp,,,,,,,,,
2022-02-15 12:59:00-06:00,b,MSFT,298.94,298.9800,299.006,298.9300,21606,456,298.970346
2022-02-15 13:00:00-06:00,b,MSFT,299.01,299.1000,299.105,298.9400,44096,816,299.033441
2022-02-15 13:01:00-06:00,b,MSFT,299.09,298.9400,299.090,298.8500,30277,560,298.939871
2022-02-15 13:02:00-06:00,b,MSFT,298.94,298.8000,298.940,298.6700,39193,680,298.785090
2022-02-15 13:03:00-06:00,b,MSFT,298.81,298.7000,298.910,298.6000,49387,866,298.739768
2022-02-15 13:04:00-06:00,b,MSFT,298.71,298.8200,298.870,298.6900,40825,675,298.776884
2022-02-15 13:05:00-06:00,b,MSFT,298.85,298.5650,298.880,298.4627,34817,734,298.617803
2022-02-15 13:06:00-06:00,b,MSFT,298.59,298.3801,298.630,298.3500,25911,576,298.469432


In [ ]:
df_unique.index = df_unique.index.tz_convert(eastern)

TypeError: ignored